In [ ]:
import re
from datetime import datetime

class User:
    def __init__(self, first_name, last_name, email, password, mobile_phone):
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.password = password
        self.mobile_phone = mobile_phone

    def __str__(self):
        return f"{self.first_name},{self.last_name},{self.email},{self.password},{self.mobile_phone}"


class Project:
    def __init__(self, title, details, total_target, start_date, end_date, owner_email):
        self.title = title
        self.details = details
        self.total_target = total_target
        self.start_date = start_date
        self.end_date = end_date
        self.owner_email = owner_email

    def __str__(self):
        return f"{self.title},{self.details},{self.total_target},{self.start_date},{self.end_date},{self.owner_email}"


class AuthenticationSystem:
    def __init__(self, filename="users.txt", projects_file="projects.txt"):
        self.filename = filename
        self.projects_file = projects_file
        self.users = self._load_users()
        self.projects = self._load_projects()

    def _load_users(self):
        users = {}
        try:
            with open(self.filename, "r") as file:
                for line in file:
                    first_name, last_name, email, password, mobile_phone = line.strip().split(",")
                    users[email] = User(first_name, last_name, email, password, mobile_phone)
        except FileNotFoundError:
            pass
        return users

    def _save_users(self):
        with open(self.filename, "w") as file:
            for user in self.users.values():
                file.write(str(user) + "\n")

    def _load_projects(self):
        projects = []
        try:
            with open(self.projects_file, "r") as file:
                for line in file:
                    title, details, total_target, start_date, end_date, owner_email = line.strip().split(",")
                    projects.append(Project(title, details, total_target, start_date, end_date, owner_email))
        except FileNotFoundError:
            pass
        return projects

    def _save_projects(self):
        with open(self.projects_file, "w") as file:
            for project in self.projects:
                file.write(str(project) + "\n")

    def _validate_egyptian_phone(self, phone):
        pattern = r"^01[0125][0-9]{8}$"
        return re.match(pattern, phone) is not None

    def _validate_email(self, email):
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return re.match(pattern, email) is not None

    def _validate_date(self, date_str):
        try:
            datetime.strptime(date_str, "%Y-%m-%d")
            return True
        except ValueError:
            return False

    def register(self):
        print("Registration:")
        first_name = input("First Name: ")
        last_name = input("Last Name: ")
        email = input("Email: ")

     
        while not self._validate_email(email):
            print("Invalid email format. Please try again.")
            email = input("Email: ")

      
        if email in self.users:
            print("Email already registered. Please login instead.")
            return

        password = input("Password: ")
        confirm_password = input("Confirm Password: ")

     
        while password != confirm_password:
            print("Passwords do not match. Please try again.")
            password = input("Password: ")
            confirm_password = input("Confirm Password: ")

        mobile_phone = input("Mobile Phone (Egyptian number): ")

        
        while not self._validate_egyptian_phone(mobile_phone):
            print("Invalid Egyptian phone number. Please try again.")
            mobile_phone = input("Mobile Phone (Egyptian number): ")

        
        new_user = User(first_name, last_name, email, password, mobile_phone)

        
        self.users[email] = new_user
        self._save_users()
        print("Registration successful! You can now login.")

    def login(self):
        print("Login:")
        email = input("Email: ")
        password = input("Password: ")

        if email in self.users and self.users[email].password == password:
            print(f"Welcome, {self.users[email].first_name} {self.users[email].last_name}!")
            self.user_menu(email)
        else:
            print("Invalid email or password.")

    def user_menu(self, email):
        while True:
            print("\nUser Menu:")
            print("1. Create Project")
            print("2. View All Projects")
            print("3. Edit Your Projects")
            print("4. Delete Your Project")
            print("5. Search Projects by Date")
            print("6. Logout")
            choice = input("Choose an option: ")

            if choice == '1':
                self.create_project(email)
            elif choice == '2':
                self.view_all_projects()
            elif choice == '3':
                self.edit_project(email)
            elif choice == '4':
                self.delete_project(email)
            elif choice == '5':
                self.search_projects_by_date()
            elif choice == '6':
                print("Logging out...")
                break
            else:
                print("Invalid choice. Please try again.")

    def create_project(self, email):
        print("Create Project:")
        title = input("Title: ")
        details = input("Details: ")
        total_target = input("Total Target (EGP): ")

        start_date = input("Start Date (YYYY-MM-DD): ")
        while not self._validate_date(start_date):
            print("Invalid date format. Please use YYYY-MM-DD.")
            start_date = input("Start Date (YYYY-MM-DD): ")

        end_date = input("End Date (YYYY-MM-DD): ")
        while not self._validate_date(end_date):
            print("Invalid date format. Please use YYYY-MM-DD.")
            end_date = input("End Date (YYYY-MM-DD): ")

  
        new_project = Project(title, details, total_target, start_date, end_date, email)
        self.projects.append(new_project)
        self._save_projects()
        print("Project created successfully!")

    def view_all_projects(self):
        print("\nAll Projects:")
        for project in self.projects:
            print(f"Title: {project.title}, Details: {project.details}, Target: {project.total_target} EGP, "
                  f"Start Date: {project.start_date}, End Date: {project.end_date}, Owner: {project.owner_email}")

    def edit_project(self, email):
        print("\nYour Projects:")
        user_projects = [project for project in self.projects if project.owner_email == email]
        for i, project in enumerate(user_projects):
            print(f"{i + 1}. Title: {project.title}, Details: {project.details}, Target: {project.total_target} EGP, "
                  f"Start Date: {project.start_date}, End Date: {project.end_date}")

        if not user_projects:
            print("You have no projects to edit.")
            return

        try:
            choice = int(input("Select a project to edit (by number): ")) - 1
            if 0 <= choice < len(user_projects):
                project = user_projects[choice]
                print(f"Editing Project: {project.title}")
                project.title = input("New Title: ")
                project.details = input("New Details: ")
                project.total_target = input("New Total Target (EGP): ")

                start_date = input("New Start Date (YYYY-MM-DD): ")
                while not self._validate_date(start_date):
                    print("Invalid date format. Please use YYYY-MM-DD.")
                    start_date = input("New Start Date (YYYY-MM-DD): ")
                project.start_date = start_date

                end_date = input("New End Date (YYYY-MM-DD): ")
                while not self._validate_date(end_date):
                    print("Invalid date format. Please use YYYY-MM-DD.")
                    end_date = input("New End Date (YYYY-MM-DD): ")
                project.end_date = end_date

                self._save_projects()
                print("Project updated successfully!")
            else:
                print("Invalid selection.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    def delete_project(self, email):
        print("\nYour Projects:")
        user_projects = [project for project in self.projects if project.owner_email == email]
        for i, project in enumerate(user_projects):
            print(f"{i + 1}. Title: {project.title}, Details: {project.details}, Target: {project.total_target} EGP, "
                  f"Start Date: {project.start_date}, End Date: {project.end_date}")

        if not user_projects:
            print("You have no projects to delete.")
            return

        try:
            choice = int(input("Select a project to delete (by number): ")) - 1
            if 0 <= choice < len(user_projects):
                project = user_projects[choice]
                self.projects.remove(project)
                self._save_projects()
                print("Project deleted successfully!")
            else:
                print("Invalid selection.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    def search_projects_by_date(self):
        date = input("Enter date to search (YYYY-MM-DD): ")
        while not self._validate_date(date):
            print("Invalid date format. Please use YYYY-MM-DD.")
            date = input("Enter date to search (YYYY-MM-DD): ")

        found_projects = [project for project in self.projects if project.start_date <= date <= project.end_date]
        if found_projects:
            print("\nProjects Found:")
            for project in found_projects:
                print(f"Title: {project.title}, Details: {project.details}, Target: {project.total_target} EGP, "
                      f"Start Date: {project.start_date}, End Date: {project.end_date}, Owner: {project.owner_email}")
        else:
            print("No projects found for the given date.")

    def main_menu(self):
        while True:
            print("\nMain Menu:")
            print("1. Register")
            print("2. Login")
            print("3. Exit")
            choice = input("Choose an option: ")

            if choice == '1':
                self.register()
            elif choice == '2':
                self.login()
            elif choice == '3':
                print("Exiting...")
                break
            else:
                print("Invalid choice. Please try again.")

auth_system = AuthenticationSystem()
auth_system.main_menu()


Main Menu:
1. Register
2. Login
3. Exit


Choose an option:  1


Registration:


First Name:  reem
Last Name:  mojam
Email:  aa@dd.com
Password:  aaaa
Confirm Password:  aaa


Passwords do not match. Please try again.


...

Project created successfully!

User Menu:
1. Create Project
2. View All Projects
3. Edit Your Projects
4. Delete Your Project
5. Search Projects by Date
6. Logout


Choose an option:  5
Enter date to search (YYYY-MM-DD):  2000-03-02



Projects Found:
Title: qqq, Details: www, Target: 1222 EGP, Start Date: 1999-02-03, End Date: 2024-04-05, Owner: aa@dd.com

User Menu:
1. Create Project
2. View All Projects
3. Edit Your Projects
4. Delete Your Project
5. Search Projects by Date
6. Logout


Choose an option:  5
Enter date to search (YYYY-MM-DD):  1999-02-03



Projects Found:
Title: qqq, Details: www, Target: 1222 EGP, Start Date: 1999-02-03, End Date: 2024-04-05, Owner: aa@dd.com

User Menu:
1. Create Project
2. View All Projects
3. Edit Your Projects
4. Delete Your Project
5. Search Projects by Date
6. Logout
